In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
X = np.loadtxt('lista01/ImageRawReduced.txt')  # 3 x 9350
C = np.loadtxt('lista01/ImageExpertReduced.txt', dtype=int)  # 1 x 9350
Rules = np.loadtxt('lista01/ClassificationRules.txt', dtype=int)  # 266 x 9350
full_C = np.loadtxt('lista01/ImageExpert.txt', dtype=int)  # 1 x 15688

/var/folders/71/db_g09c51311049r8gdj12m00000gn/T/ipykernel_90301/2439081656.py:2: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  C = np.loadtxt('lista01/ImageExpertReduced.txt', dtype=int)  # 1 x 9350
/var/folders/71/db_g09c51311049r8gdj12m00000gn/T/ipykernel_90301/2439081656.py:3: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `

In [3]:
def evaluate_weight_vector(weight_vector, Rules, C):
    weighted_votes = Rules * weight_vector[:, None]
    class_votes = np.zeros((3, Rules.shape[1]))

    for j in range(1, 4):
        class_votes[j-1] = np.sum(weighted_votes * (Rules == j), axis=0)
    
    predictions = np.argmax(class_votes, axis=0) + 1 #+1 for class labels [1, 2, 3]

    correct_predictions = np.sum(predictions == C)
    accuracy = correct_predictions / len(C) * 100 

    return accuracy


In [13]:
def simulated_annealing(Rules, C, max_iter=1000, initial_temp=100, cooling_rate=0.99, alpha=0.001, perturbation_size=0.05):
    d = Rules.shape[0]  
    
    current_weights = np.random.rand(d)
    current_accuracy = evaluate_weight_vector(current_weights, Rules, C)
    
    best_weights = np.copy(current_weights)
    best_accuracy = current_accuracy
    
    T = initial_temp 
    
    for i in range(max_iter):
        new_weights = np.copy(current_weights)
        perturbation = (np.random.rand(d) - 0.5) * perturbation_size * 2  
        new_weights = np.clip(new_weights + perturbation, 0, 1)  
        
        new_accuracy = evaluate_weight_vector(new_weights, Rules, C)
        
        delta_accuracy = new_accuracy - current_accuracy
        
        if delta_accuracy > 0:
            current_weights = new_weights
            current_accuracy = new_accuracy
            
            if new_accuracy > best_accuracy:
                best_weights = new_weights
                best_accuracy = new_accuracy
        else:
            acceptance_probability = np.exp(-alpha * abs(delta_accuracy) / T)
            if np.random.rand() < acceptance_probability:
                current_weights = new_weights
                current_accuracy = new_accuracy
        
        T *= cooling_rate
        
        if (i + 1) % 100 == 0:
            print(f"Iteration {i+1}/{max_iter}, Current Accuracy: {current_accuracy:.2f}%, Best Accuracy: {best_accuracy:.2f}%")
    
    return best_weights, best_accuracy

In [15]:
best_weight_vector, best_accuracy = simulated_annealing(Rules, C, max_iter=10000, initial_temp=100, cooling_rate=0.99)
print(best_weight_vector)
print(best_accuracy)

Iteration 100/10000, Current Accuracy: 68.29%, Best Accuracy: 69.27%
Iteration 200/10000, Current Accuracy: 70.71%, Best Accuracy: 72.22%
Iteration 300/10000, Current Accuracy: 71.79%, Best Accuracy: 72.22%
Iteration 400/10000, Current Accuracy: 71.80%, Best Accuracy: 73.12%
Iteration 500/10000, Current Accuracy: 68.89%, Best Accuracy: 73.12%
Iteration 600/10000, Current Accuracy: 57.21%, Best Accuracy: 73.12%
Iteration 700/10000, Current Accuracy: 66.11%, Best Accuracy: 73.12%
Iteration 800/10000, Current Accuracy: 66.35%, Best Accuracy: 73.12%
Iteration 900/10000, Current Accuracy: 72.50%, Best Accuracy: 73.12%
Iteration 1000/10000, Current Accuracy: 70.06%, Best Accuracy: 73.12%
Iteration 1100/10000, Current Accuracy: 72.18%, Best Accuracy: 73.12%
Iteration 1200/10000, Current Accuracy: 71.86%, Best Accuracy: 73.12%
Iteration 1300/10000, Current Accuracy: 75.20%, Best Accuracy: 75.22%
Iteration 1400/10000, Current Accuracy: 78.25%, Best Accuracy: 78.25%
Iteration 1500/10000, Current

In [16]:
best_weight_vector, best_accuracy = simulated_annealing(Rules, C, max_iter=100000, initial_temp=100, cooling_rate=0.99)
print(best_weight_vector)
print(best_accuracy)

Iteration 100/100000, Current Accuracy: 71.49%, Best Accuracy: 74.05%
Iteration 200/100000, Current Accuracy: 70.80%, Best Accuracy: 74.05%
Iteration 300/100000, Current Accuracy: 70.87%, Best Accuracy: 74.05%
Iteration 400/100000, Current Accuracy: 71.52%, Best Accuracy: 74.05%
Iteration 500/100000, Current Accuracy: 71.44%, Best Accuracy: 74.05%
Iteration 600/100000, Current Accuracy: 70.41%, Best Accuracy: 74.05%
Iteration 700/100000, Current Accuracy: 66.65%, Best Accuracy: 74.05%
Iteration 800/100000, Current Accuracy: 65.75%, Best Accuracy: 74.05%
Iteration 900/100000, Current Accuracy: 51.25%, Best Accuracy: 74.05%
Iteration 1000/100000, Current Accuracy: 64.91%, Best Accuracy: 74.05%
Iteration 1100/100000, Current Accuracy: 71.41%, Best Accuracy: 74.05%
Iteration 1200/100000, Current Accuracy: 72.47%, Best Accuracy: 74.05%
Iteration 1300/100000, Current Accuracy: 75.65%, Best Accuracy: 76.13%
Iteration 1400/100000, Current Accuracy: 79.71%, Best Accuracy: 79.71%
Iteration 1500/

C:\Users\szcze\AppData\Local\Temp\ipykernel_32396\3745440962.py:52: RuntimeWarning: overflow encountered in scalar divide
  acceptance_probability = np.exp(-alpha * abs(delta_accuracy) / T)


Iteration 71900/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72000/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72100/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72200/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72300/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72400/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72500/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72600/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72700/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72800/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 72900/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 73000/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 73100/100000, Current Accuracy: 98.22%, Best Accuracy: 98.22%
Iteration 73200/100000, Current Accuracy: 98.22%, Best Accuracy:

PBIL try

In [23]:
n_iter = 100
pop_size = 500
d = 266  
lr = 0.15 
nr = 0.03  
mut_rate = 0.02  
mut_size = 0.03  

In [24]:
P = np.full(d, 0.5) 
best_fitness_over_time = []
for iter in range(n_iter):
    population = np.random.rand(pop_size, d) #each individual is now vector of weights instead of binary vector
    fitness = np.zeros(pop_size)
    
    for i in range(pop_size):
        individual = population[i]
        
        weighted_votes = Rules * individual[:, None]

        class_votes = np.zeros((3, Rules.shape[1]))
        
        for j in range(1, 4):
            class_votes[j-1, :] = np.sum(weighted_votes * (Rules == j), axis=0)
                
        predictions = np.argmax(class_votes, axis=0) + 1
        
        fitness[i] = np.sum(predictions == C)
    
    best_idx = np.argmax(fitness)
    worst_idx = np.argmin(fitness)
    best_individual = population[best_idx]
    worst_individual = population[worst_idx]
    
    P = P * (1 - lr) + best_individual * lr
    P = P * (1 - nr) + worst_individual * nr
    
    mutation = np.random.rand(d) < mut_rate
    P[mutation] = P[mutation] * (1 - mut_size) + np.random.rand(np.sum(mutation)) * mut_size
    
    P = np.clip(P, 0, 1)
    
    best_accuracy = fitness[best_idx] / len(C) * 100
    best_fitness_over_time.append(best_accuracy)

    print(f"Iteracja {iter+1}/{n_iter}, najlepsza dokładność: {best_accuracy:.2f}%")

Iteracja 1/100, najlepsza dokładność: 75.86%
Iteracja 2/100, najlepsza dokładność: 77.05%
Iteracja 3/100, najlepsza dokładność: 76.18%
Iteracja 4/100, najlepsza dokładność: 76.50%
Iteracja 5/100, najlepsza dokładność: 76.15%
Iteracja 6/100, najlepsza dokładność: 76.15%
Iteracja 7/100, najlepsza dokładność: 75.32%
Iteracja 8/100, najlepsza dokładność: 75.60%
Iteracja 9/100, najlepsza dokładność: 75.94%
Iteracja 10/100, najlepsza dokładność: 75.86%
Iteracja 11/100, najlepsza dokładność: 77.36%
Iteracja 12/100, najlepsza dokładność: 76.92%
Iteracja 13/100, najlepsza dokładność: 75.79%
Iteracja 14/100, najlepsza dokładność: 75.79%
Iteracja 15/100, najlepsza dokładność: 76.86%
Iteracja 16/100, najlepsza dokładność: 76.04%
Iteracja 17/100, najlepsza dokładność: 76.40%
Iteracja 18/100, najlepsza dokładność: 75.76%
Iteracja 19/100, najlepsza dokładność: 76.03%
Iteracja 20/100, najlepsza dokładność: 76.58%
Iteracja 21/100, najlepsza dokładność: 75.42%
Iteracja 22/100, najlepsza dokładność: 75.8

KeyboardInterrupt: 

In [19]:
print(best_individual)

[0.05991105 0.81938966 0.19962738 0.39042555 0.97819318 0.98608374
 0.90378012 0.99169331 0.73864884 0.41980865 0.24874232 0.85791827
 0.37006029 0.25719066 0.28871644 0.8458935  0.99520123 0.27598206
 0.72985528 0.74877788 0.18377297 0.53233427 0.17501388 0.63839347
 0.42433122 0.94107496 0.7227856  0.31972671 0.65299929 0.76919015
 0.11720313 0.61986764 0.01326282 0.52758683 0.09211967 0.41076214
 0.97116488 0.82463353 0.80045417 0.66742901 0.24284737 0.50728528
 0.14038291 0.19870398 0.91745076 0.18245205 0.26098489 0.19130494
 0.86554166 0.10644329 0.76346258 0.52456106 0.45284862 0.8014676
 0.24153604 0.17844516 0.03910753 0.0204148  0.00908909 0.62369934
 0.57649763 0.19989123 0.13712906 0.17885003 0.69653396 0.57026943
 0.9871855  0.62919072 0.21907572 0.94383188 0.71030123 0.67624757
 0.87315481 0.84732126 0.28661942 0.16919363 0.87086686 0.4535131
 0.02845797 0.93221357 0.83989244 0.63255975 0.50058844 0.31927678
 0.9814378  0.48819112 0.67672211 0.6234815  0.38331173 0.203665